# Before your begin

1. Setup **gcloud** by following this guide [here](https://29022131.atlassian.net/wiki/spaces/DP/pages/1006174505/JupyterHub+-+End-user+Guide#JupyterHub-End-userGuide-GCloudsetup).
2. Setup **github** by following this guide [here](https://29022131.atlassian.net/wiki/spaces/DP/pages/1006174505/JupyterHub+-+End-user+Guide#JupyterHub-End-userGuide-Githubsetup).

# Export BQ data to GCS to Pandas Dataframe

Usually, a convenient way to convert BQ query result to pandas dataframe is to use to_dataframe() method. However, if your data is large, it will become significantly slower.
One way to tackle this problem is to export the query result to another BQ temp table, then export this table to GCS in CSV format, then load it to pandas dataframe.

In [ ]:
from google.cloud import storage
from google.cloud import bigquery
import pandas as pd

#### Initiate bq client

In [ ]:
client = bigquery.Client(project="YOUR-PROJECT-ID")

#### Sample query

In [ ]:
query = (
        "SELECT "
        "trip_distance, fare_amount "
        "FROM "
        "`nyc-tlc.yellow.trips` "
        "WHERE "
        "trip_distance > 0 "
        "AND passenger_count > 0 "
        "AND fare_amount >= 2.5 "
        "AND pickup_longitude > -78 "
        "AND pickup_longitude < -70 "
        "AND dropoff_longitude > -78 "
        "AND dropoff_longitude < -70 "
        "AND pickup_latitude > 37 "
        "AND pickup_latitude < 45 "
        "AND dropoff_latitude > 37 "
        "AND dropoff_latitude < 45 "
        "AND MOD(ABS(FARM_FINGERPRINT(CAST(pickup_datetime AS STRING))), 5000) = 1"
    )

#### Initiate bq job config

In [ ]:
dataset_name = "YOUR-DATASET-NAME"

In [ ]:
table_name = "YOUR-TEMP-TABLE-NAME"

In [ ]:
job_config = bigquery.QueryJobConfig()
table_ref = client.dataset(dataset_name).table(table_name)
job_config.destination = table_ref
# by default it will truncate the table if exists
job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

#### Execute the query and save the result to another BQ temp table (configured in the job_config above)

In [ ]:
query_job = client.query(
    query,
    location="US",
    job_config=job_config)

#### Extract the table to a CSV file in GCS

In [ ]:
destination_uri = "gs://YOUR-BUCKET/path/to/your/file.csv"

In [ ]:
extract_job = client.extract_table(
    table_ref,
    destination_uri,
    location='US')

#### Load CSV file in GCS to Pandas Dataframe

In [ ]:
!pip install gcsfs

In [ ]:
df = pd.read_csv(destination_uri)

In [ ]:
df

#### Deleting BQ temp table and GCS file

In [ ]:
client.delete_table(table_ref)

In [ ]:
storage_client = storage.Client()
bucket = storage_client.get_bucket("YOUR-BUCKET")
blob = bucket.blob("path/to/your/file.csv")

blob.delete()